In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import scipy
import torch
from numpy import genfromtxt
from matplotlib import pyplot as plt
# initialize variables
N = 10 # number of sites
half_filling = True

if half_filling == True:

    N_e = N # total number of electrons
    S_z = 0
W = 2*N # each sites can have 2 electrons
# Q is the dimension
Q = pow(4, N)

# decimal number to binary array function
def D2B(num):
    string = f'{num:1b}'
    result = np.zeros(W-len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result #nd array


# binary array to decimal function
def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele
    return res[0]

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def scipy_to_torch_sparse(scipy_coo_matrix):
    """
    Convert a SciPy sparse COO matrix to a PyTorch sparse COO matrix.

    Parameters:
    scipy_coo_matrix (scipy.sparse.coo_matrix): The input SciPy sparse COO matrix.

    Returns:
    torch.sparse_coo_tensor: The converted PyTorch sparse COO tensor.
    """
    if not isinstance(scipy_coo_matrix,scipy.sparse.coo_matrix):
        raise TypeError("Input matrix must be a SciPy sparse COO matrix")

    # Extract row, column, and data
    row = torch.tensor(scipy_coo_matrix.row, dtype=torch.long)
    col = torch.tensor(scipy_coo_matrix.col, dtype=torch.long)
    data = torch.tensor(scipy_coo_matrix.data)

    # Create the indices tensor
    indices = torch.stack([row, col], dim=0)

    # Create the PyTorch sparse COO tensor
    torch_sparse_tensor = torch.sparse_coo_tensor(indices, data, scipy_coo_matrix.shape, dtype=torch.complex128)

    return torch_sparse_tensor

In [ ]:
# load  data
file_path = f'/content/drive/My Drive/Hubbard_2024/2d/old_index_list={N}.txt'
old_index_list = np.loadtxt(file_path, dtype=int)

In [ ]:
U_value = 2
# Load the sparse matrix in pytorch COO format with complex 128 data
h_sparse = scipy.sparse.load_npz(f'/content/drive/My Drive/Hubbard_2024/2d/es/torch_h_sparse_N_{N}_U_{U_value}.npz')
h_sparse = scipy_to_torch_sparse(h_sparse)
h_sparse = h_sparse.to(device)
# Load the eigenvalues
true_gs_energy = np.loadtxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/true_gs_energy_N_{N}_U_{U_value}.csv', delimiter=',')

# Load the eigenvectors
true_gs = np.loadtxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/true_gs_N_{N}_U_{U_value}.csv', delimiter=',')
# move to gpu
true_gs = torch.tensor(true_gs, dtype=torch.complex128).to(device)
# definite spin matrix
#def_spin_matrix = sparse.load_npz(f'/content/drive/My Drive/Hubbard_2024/2d/definite_spin_matrix_N={N}.npz')


In [ ]:
print(true_gs_energy)

-9.508902323907462


In [ ]:
def calculate_energy(input_matrix, psi): # input_matrix is sparse
    psi_dagger = torch.conj(psi)  # Conjugate of the state vector

    # Sparse matrix-vector multiplication
    intermediate = torch.sparse.mm(input_matrix, psi.unsqueeze(1))  # shape (n, 1)

    # Dot product to get the energy
    energy = torch.dot(psi_dagger, intermediate.squeeze())  # shape (n,)

    return energy.real

In [ ]:
print(calculate_energy(h_sparse,true_gs))

tensor(-9.5089, device='cuda:0', dtype=torch.float64)


In [ ]:
# test code to check calculate eneryg is correct
# print(calculate_energy(h_sparse,true_gs)-true_gs_energy)

In [ ]:
random_state = False
state_to_evolve = torch.zeros(len(true_gs), dtype=torch.complex128)

# Set up PyTorch random number generator
torch.manual_seed(0)  # Set a fixed random seed for reproducibility

# Initialize state_to_evolve based on random_state and N
if random_state:
    state_to_evolve_real = torch.rand(len(true_gs), dtype=torch.float64) * 2 - 1
    state_to_evolve_imag = torch.rand(len(true_gs), dtype=torch.float64) * 2 - 1
    state_to_evolve = state_to_evolve_real + 1j * state_to_evolve_imag
else:
    if N == 8:
        state_to_evolve[4005] = 1
    elif N == 4:
        state_to_evolve[5] = 1
    elif N == 6:
        if U_value == 8:
            state_to_evolve[64] = 1
        else:
            state_to_evolve[70] = 1
            state_to_evolve[71] = 1
    elif N == 10:
        if U_value == 8:

            state_to_evolve[51633] = 1
        if U_value == 2:
            state_to_evolve[11871] = 1
            state_to_evolve[11872] = 1

    elif N == 12:
        state_to_evolve[691965] = 1 # or index 161810

# Normalize the state vector
state_to_evolve = state_to_evolve / torch.norm(state_to_evolve)

# move to gpu
state_to_evolve = state_to_evolve.to(device)


In [ ]:
# # if start with U=0 ground state

# # Load the eigenvectors
# state_to_evolve = np.loadtxt(f'/content/drive/My Drive/Hubbard_2024/2d/true_gs_N_{N}_U_{0}.csv', delimiter=',')
# # move to gpu
# state_to_evolve = torch.tensor(state_to_evolve, dtype=torch.complex128).to(device)

In [ ]:
def calculate_fidelity(psi, phi):
    """
    Calculate the fidelity between two state vectors.

    Parameters:
    psi (torch.Tensor): The first state vector in PyTorch format, dtype=torch.complex128.
    phi (torch.Tensor): The second state vector in PyTorch format, dtype=torch.complex128.

    Returns:
    float: The calculated fidelity.
    """
    fidelity = torch.abs(torch.vdot(psi, phi))**2
    return fidelity.item()

In [ ]:
calculate_fidelity(state_to_evolve,true_gs)

1.135674130993585e-07

In [ ]:
# # check the energy of the state to evolve
print(calculate_energy(h_sparse,state_to_evolve))

tensor(8.0000, device='cuda:0', dtype=torch.float64)


In [ ]:
# U part, read data
file_path = f'/content/drive/My Drive/Hubbard_2024/2d/number_op_list_N={N}.txt'
number_op_list = torch.tensor(np.loadtxt(file_path))
# move to gpu
number_op_list = number_op_list.to(device)

In [ ]:
def UCC_U_part_torch(input_state, para_index, input_theta):
    """
    Apply the UCC U part operation on the input state.

    Parameters:
    input_state (torch.Tensor): The input state tensor in PyTorch format, dtype=torch.complex128.
    para_index (int): The index to select the corresponding number operator from number_op_list.
    input_theta (float): The input theta parameter for the phase factor.

    Returns:
    torch.Tensor: The output state after applying the phase factors.
    """
    occ_list = number_op_list[para_index]

    # Compute the phase factors
    phase_factor_list = torch.exp(1j * input_theta * occ_list)

    # Apply the phase factors to the input state
    output_state = phase_factor_list * input_state

    return output_state


In [ ]:
# UCC_U_part_torch(state_to_evolve,2,torch.tensor([0.3],dtype=torch.float64).to(device ))

In [ ]:
import pickle
# Specify the file path to save the dictionary
file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_hopping_cose_dict_N={N}.pkl'

# Load the dictionary from the file
hopping_cose_dict = pickle.load(open(file_path, 'rb'))

# Convert numpy arrays to torch tensors
hopping_cose_dict = {key: torch.tensor(value, device=device) for key, value in hopping_cose_dict.items()}

file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_sparse_matrices_dict_N={N}.pkl'

# Load the dictionary from the file
sparse_matrices_dict = pickle.load(open(file_path, 'rb'))

# Convert SciPy sparse COO matrices to PyTorch sparse COO tensors
sparse_matrices_dict = {key: scipy_to_torch_sparse(value) for key, value in sparse_matrices_dict.items()}
# Move PyTorch sparse COO tensors to GPU
sparse_matrices_dict = {key: value.to(device) for key, value in sparse_matrices_dict.items()}


In [ ]:
def UCC_t_part_torch(input_state, hopping_pair_site, ri_sign, input_theta):
    """
    Apply the UCC t part operation on the input state.

    Parameters:
    input_state (torch.Tensor): The input state tensor in PyTorch format, dtype=torch.complex128.
    hopping_pair_site (tuple): The hopping pair site indices.
    ri_sign (int): The sign indicating the type of sine part calculation.
    input_theta (float): The input theta parameter for the phase factor.

    Returns:
    torch.Tensor: The output state after applying the phase factors.
    """
    # Loop over spins since both need to be applied
    for spin in [1, -1]:
        # Define the hopping pair including the spin
        hopping_pair = hopping_pair_site + (spin,)

        # Cosine part
        cos_list = torch.ones(len(true_gs), dtype=torch.complex128).to(device)
        cos_list_index = hopping_cose_dict[hopping_pair]
        cos_list[cos_list_index] = torch.cos(input_theta).to(torch.complex128)
        part1 = cos_list * input_state

        # Sine part
        input_state = input_state.view(-1, 1)
        if ri_sign == 1:
            hop_matrix = sparse_matrices_dict[hopping_pair] - sparse_matrices_dict[hopping_pair].transpose(0, 1)
            part2 = torch.sparse.mm(hop_matrix, input_state).view(-1)
            final_state = part1 + torch.sin(input_theta).to(torch.complex128) * part2
        else:
            hop_matrix = sparse_matrices_dict[hopping_pair] + sparse_matrices_dict[hopping_pair].transpose(0, 1)
            part2 =torch.sparse.mm(hop_matrix, input_state).view(-1)
            final_state = part1 + 1j * torch.sin(input_theta).to(torch.complex128) * part2

        # Update the state for spin down, i.e., -1
        input_state = final_state

    return final_state

In [ ]:
## numerical check using actual matrix exponetial, real part

# SU2 method
# UCC_t_part_torch(state_to_evolve, (0,3),-1,torch.tensor([0.3],dtype=torch.float64).to(device ))
# # numerical
# b = sparse_matrices_dict[(0,3,1)]+ sparse_matrices_dict[(0,3,1)].T+ sparse_matrices_dict[(0,3,-1)]+ sparse_matrices_dict[(0,3,-1)].T
# b = b.to_dense()
# torch.matrix_exp(b)
# torch.matmul(torch.matrix_exp(b),state_to_evolve) - UCC_t_part_torch(state_to_evolve, (0,1),-1,torch.tensor([0.3],dtype=torch.complex128).to(device ))

In [ ]:
# Define parameters
para_repeat_times = 23
para_value_list = []
counting_parameter_index = 0
nn_condition = True  # True for nearest-neighbor hopping

# Step 1: Generate hopping pairs and U terms for each repeat time
for _ in range(para_repeat_times):
    hopping_terms = []
    U_terms = []

    # Generate hopping terms
    for i in range(N):
        for j in range(N):
            if nn_condition:
                if j == i + 1:
                    hopping_terms.append((i, j, -1, counting_parameter_index))
                    counting_parameter_index += 1
            else:
                if j > i:
                    hopping_terms.append((i, j, -1, counting_parameter_index))
                    counting_parameter_index += 1

    # Generate U terms
    for i in range(N):
        U_terms.append((i, counting_parameter_index))
        counting_parameter_index += 1

    # Step 2: Group commuting terms
    def can_commute(term, group):
        # A term can commute with a group if none of the sites in the term are present in any terms in the group
        term_sites = set(term[:2])  # Extract the two sites involved in the hopping term
        for existing_term in group:
            existing_sites = set(existing_term[:2])  # Extract sites from existing terms
            if term_sites & existing_sites:  # Check if there is any overlap in sites
                return False  # Term cannot commute with this group
        return True  # Term can commute with this group

    # Initialize an empty list to hold groups of commuting terms
    commuting_groups = []

    # Sort hopping terms into commuting groups
    for term in hopping_terms:
        added = False
        for group in commuting_groups:
            if can_commute(term, group):
                group.append(term)  # Add term to the group if it commutes with all terms in the group
                added = True
                break
        if not added:
            commuting_groups.append([term])  # Start a new group if it doesn't commute with any existing group

    # Step 3: Flatten the list of groups into the final ordered list (only for hopping terms)
    ordered_hopping_list = [term for group in commuting_groups for term in group]

    # Step 4: Append the hopping terms and U terms in the correct order
    para_value_list.extend(ordered_hopping_list)
    para_value_list.extend(U_terms)

# Return the final list
print("Final para_value_list with commuting groups and U terms ordered:")
print(len(para_value_list), para_value_list)

# Optional: You can also print the groups if you'd like to visualize them
print("\nCommuting groups per repeat:")
for idx, group in enumerate(commuting_groups):
    print(f"Group {idx+1}: {group}")

Final para_value_list with commuting groups and U terms ordered:
437 [(0, 1, -1, 0), (2, 3, -1, 2), (4, 5, -1, 4), (6, 7, -1, 6), (8, 9, -1, 8), (1, 2, -1, 1), (3, 4, -1, 3), (5, 6, -1, 5), (7, 8, -1, 7), (0, 9), (1, 10), (2, 11), (3, 12), (4, 13), (5, 14), (6, 15), (7, 16), (8, 17), (9, 18), (0, 1, -1, 19), (2, 3, -1, 21), (4, 5, -1, 23), (6, 7, -1, 25), (8, 9, -1, 27), (1, 2, -1, 20), (3, 4, -1, 22), (5, 6, -1, 24), (7, 8, -1, 26), (0, 28), (1, 29), (2, 30), (3, 31), (4, 32), (5, 33), (6, 34), (7, 35), (8, 36), (9, 37), (0, 1, -1, 38), (2, 3, -1, 40), (4, 5, -1, 42), (6, 7, -1, 44), (8, 9, -1, 46), (1, 2, -1, 39), (3, 4, -1, 41), (5, 6, -1, 43), (7, 8, -1, 45), (0, 47), (1, 48), (2, 49), (3, 50), (4, 51), (5, 52), (6, 53), (7, 54), (8, 55), (9, 56), (0, 1, -1, 57), (2, 3, -1, 59), (4, 5, -1, 61), (6, 7, -1, 63), (8, 9, -1, 65), (1, 2, -1, 58), (3, 4, -1, 60), (5, 6, -1, 62), (7, 8, -1, 64), (0, 66), (1, 67), (2, 68), (3, 69), (4, 70), (5, 71), (6, 72), (7, 73), (8, 74), (9, 75), (0, 

In [ ]:
# # also save the ops list
# file_path = f'/content/drive/My Drive/Hubbard_2024/2d/result/para_list_N={N}_U={U_value}_steps_{para_repeat_times}.txt'
# with open(file_path, 'w') as f:
#     for item in para_value_list:
#         f.write(f"{item}\n")

In [ ]:
# evolve state
# also get fidelity during the time evolution
def evolve_state(x):
    fidelity_list = []
    energy_list = []
    state_to_evolve_1 = state_to_evolve.clone()

    for para_index in para_value_list:
        if len(para_index) == 2:  # This is non-hopping only
            new_state = UCC_U_part_torch(state_to_evolve_1, para_index[0], x[para_index[1]])
        else:
            site_index = para_index[:2]
            rl_sign = para_index[2]
            x_index = para_index[3]
            new_state = UCC_t_part_torch(state_to_evolve_1, site_index, rl_sign, x[x_index])

        # Update state for next loop
        state_to_evolve_1 = new_state
        #fidelity_list.append(calculate_fidelity( true_gs,state_to_evolve_1))
        #energy_list.append(calculate_energy(h_sparse,state_to_evolve_1))
    return state_to_evolve_1


def classical_optimizer(x):
    """
    Perform classical optimization.
    Parameters:
    x (torch.Tensor): The parameter tensor.
    state_to_evolve (torch.Tensor): The initial state tensor.
    para_value_list (list): The list of parameter indices.
    h_sparse (torch.sparse.FloatTensor or torch.sparse.DoubleTensor): The sparse Hamiltonian matrix.

    Returns:
    float64: The calculated energy.
    """
    state_to_evolve_1 = state_to_evolve.clone()

    for para_index in para_value_list:
        if len(para_index) == 2:  # This is non-hopping only
            new_state = UCC_U_part_torch(state_to_evolve_1, para_index[0], x[para_index[1]])
        else:
            site_index = para_index[:2]
            rl_sign = para_index[2]
            x_index = para_index[3]
            new_state = UCC_t_part_torch(state_to_evolve_1, site_index, rl_sign, x[x_index])

        # Update state for next loop
        state_to_evolve_1 = new_state

    new_energy = calculate_energy(h_sparse, new_state)
    #print(calculate_fidelity( true_gs,new_state))
    return new_energy

In [ ]:
 # read
# file_path = f'/content/drive/My Drive/Hubbard_2024/final_data/8_sites/para_list_N={N}_U={U_value}_steps_{para_repeat_times}.txt'
# para_value_list = []

# # Open the file and read the data
# with open(file_path, 'r') as f:
#     for line in f:
#         # Safely evaluate the line to convert it from string to tuple/list
#         item = ast.literal_eval(line.strip())
#         para_value_list.append(item)

In [ ]:
# x = torch.tensor(np.random.uniform(-0.01, 0.01, counting_parameter_index), dtype=torch.float64, device=device, requires_grad=True)
# classical_optimizer(x)

In [ ]:
# from typing_extensions import final
# import torch
# import torch.optim as optim
# import os
# import numpy as np

# lr = 0.1
# gradient_threshold = 0.001
# filename = f'/content/drive/My Drive/Hubbard_2024/2d/result/LBFGS_log_hubbard_{N}_sites_U_{U_value}_steps_{para_repeat_times}.txt'
# checkpoint_filename = f'/content/drive/My Drive/Hubbard_2024/2d/result/LBFGS_checkpoint_hubbard_{N}_sites_U_{U_value}_steps_{para_repeat_times}.pt'
# max_epochs = 50000


# # CPU version
# if os.path.isfile(checkpoint_filename):
#     # Load checkpoint with appropriate map_location
#     checkpoint = torch.load(checkpoint_filename, map_location=device)
#     x = torch.tensor(checkpoint['x'], dtype=torch.float64, device=device, requires_grad=True)

#     # Initialize L-BFGS optimizer
#     optimizer = optim.LBFGS([x], lr=lr)
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     epoch = checkpoint['epoch']
# else:
#     x = torch.tensor(np.random.uniform(-0.01, 0.01, counting_parameter_index), dtype=torch.float64, device=device, requires_grad=True)
#     optimizer = optim.LBFGS([x], lr=lr, max_iter=20, history_size=100)
#     epoch = 0

# def closure():
#     optimizer.zero_grad()
#     energy = classical_optimizer(x)
#     energy.backward()
#     return energy

# with open(filename, 'a') as f:  # Use 'a' to append to the log file if it exists
#     while epoch < max_epochs:
#         optimizer.step(closure)

#         # Log and save data
#         gradient_norm = x.grad.norm().cpu().detach().numpy()
#         if (epoch + 1) % 1 == 0:
#             result_str = f"Epoch {epoch + 1}, Energy: {classical_optimizer(x).cpu().detach().numpy():.6f}, Gradient Norm: {gradient_norm:.6f}\n"
#             f.write(result_str)
#             print(result_str, end='')

#             # Append latest x values to the end of the file
#             x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
#             f.write(f"Latest x values at epoch {epoch + 1}: {x_values}\n")

#             # Save checkpoint
#             torch.save({
#                 'epoch': epoch,
#                 'x': x.detach().cpu().numpy(),  # Save as numpy array for checkpoint
#                 'optimizer_state_dict': optimizer.state_dict(),
#             }, checkpoint_filename)

#         # Increment epoch counter
#         epoch += 1
#         f.flush()

#         # check if gradient is smaller than threshold
#         if gradient_norm < gradient_threshold:
#             break



In [ ]:
from typing_extensions import final
import torch
import torch.optim as optim
import os
import numpy as np

lr = 0.1
gradient_threshold = 0.001
filename = f'/content/drive/My Drive/Hubbard_2024/2d/result/LBFGS_log_hubbard_{N}_sites_U_{U_value}_steps_{para_repeat_times}.txt'
checkpoint_filename = f'/content/drive/My Drive/Hubbard_2024/2d/result/LBFGS_checkpoint_hubbard_{N}_sites_U_{U_value}_steps_{para_repeat_times}.pt'
max_epochs = 50000

# GPU version

# Initialize x if checkpoint does not exist
if os.path.isfile(checkpoint_filename):
    checkpoint = torch.load(checkpoint_filename)
    x = torch.tensor(checkpoint['x'], dtype=torch.float64, device=device, requires_grad=True)

    # Initialize L-BFGS optimizer
    optimizer = optim.LBFGS([x], lr=lr)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
else:
    x = torch.tensor(np.random.uniform(-0.001, 0.001, counting_parameter_index), dtype=torch.float64, device=device, requires_grad=True)
    optimizer = optim.LBFGS([x], lr=lr, max_iter=20, history_size=1000)
    epoch = 0

def closure():
    optimizer.zero_grad()
    energy = classical_optimizer(x)
    energy.backward()
    return energy

with open(filename, 'a') as f:  # Use 'a' to append to the log file if it exists
    while epoch < max_epochs:
        optimizer.step(closure)

        # Log and save data
        gradient_norm = x.grad.norm().cpu().detach().numpy()
        if (epoch + 1) % 1 == 0:
            result_str = f"Epoch {epoch + 1}, Energy: {classical_optimizer(x).cpu().detach().numpy():.6f}, Gradient Norm: {gradient_norm:.6f}\n"
            f.write(result_str)
            print(result_str, end='')

            # Append latest x values to the end of the file
            x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
            f.write(f"Latest x values at epoch {epoch + 1}: {x_values}\n")

            # Save checkpoint
            torch.save({
                'epoch': epoch,
                'x': x.detach().cpu().numpy(),  # Save as numpy array for checkpoint
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_filename)

        # L-BFGS does not use gradient threshold for stopping; you may need a custom stopping condition
        epoch += 1
        f.flush()

        # check if gradient is smaller than threshold
        if gradient_norm < gradient_threshold:
            break


In [ ]:
filename = f'/content/drive/My Drive/Hubbard_2024/2d/result/final_report_{N}_sites_U_{U_value}_steps_{para_repeat_times}.txt'

# once done write result file

#outcome result
length = len(para_value_list)

# Determine if tensor 'x' is on GPU or CPU
if x.is_cuda:
    # GPU version
    final_energy_ratio = classical_optimizer(x).detach().cpu().numpy() / true_gs_energy
else:
    # CPU version
    final_energy_ratio = classical_optimizer(x).detach().numpy() / true_gs_energy

######################################################################

final_fidelity = calculate_fidelity( true_gs,evolve_state(x))
x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
# Write results to file
with open(filename, 'w') as file:
    file.write(f"Final Energy Ratio: {final_energy_ratio}\n")
    file.write(f"Final Fidelity: {final_fidelity}\n")
    file.write(f"Optimized X Values: {x_values}\n")
    file.write(f"Length of parameter value list: {length}\n")
    file.write(f"Epoch : {epoch }\n")

In [ ]:
# Read the results from the file
with open(filename, 'r') as file:
    content = file.read()

# Print the file content
print(content)

import re
import torch

# Assuming 'content' is a string variable containing the text and 'device' is defined
x_values_match = re.search(r"Optimized X Values: (.*)", content)
if x_values_match:
  x_values_str = x_values_match.group(1)
  x_values = [float(x) for x in x_values_str.split(',')]
  x = torch.tensor(x_values, dtype=torch.float64, device=device, requires_grad=True)



In [ ]:
# filename = f'/content/drive/My Drive/Hubbard_2024/2d/final_result/final_report_{N}_sites_U_{U_value}_steps_{para_repeat_times}.txt'

# # Read the results from the file
# with open(filename, 'r') as file:
#     content = file.read()

# # Print the file content
# print(content)

# import re
# import torch

# # Assuming 'content' is a string variable containing the text and 'device' is defined
# x_values_match = re.search(r"Optimized X Values: (.*)", content)
# if x_values_match:
#   x_values_str = x_values_match.group(1)
#   x_values = [float(x) for x in x_values_str.split(',')]
#   x = torch.tensor(x_values, dtype=torch.float64, device=device, requires_grad=True)



In [ ]:
# f_list,e_list= evolve_state(x)
# plt.plot(f_list)
# plt.xlabel('Index of parameter m', fontsize=18)
# plt.ylabel('ground state\nfidelity', fontsize=18)  # Split the ylabel into two lines using \n
# plt.xticks([0,200,400,600,800],fontsize=18)  # Set x-tick label size
# plt.yticks(fontsize=18)  # Set y-tick label size
# plt.show()